In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.interpolate import interp1d
from matplotlib import rc

#for latex-rendering. (I guess with a lot of redundancy.. but it works!)
rc('text', usetex=True) #
plt.rcParams['text.usetex'] = True #this line turns "ticks" on axis into latex  (i think is it the same function as the line above. not sure. )
plt.rcParams['text.latex.unicode'] = True 
#for the size of the exported figures
plt.rcParams['figure.figsize'] = (24, 18)
rc('text.latex', preamble=r'\usepackage{cmbright}')#this line turns text in legends for instance into latex

These definitions will be use to have compact format for the "ticks" on plots.

In [2]:
def round_to_n(x, n):
    " Round x to n significant figures "
    return round(x, -int(math.floor(np.sign(x) * math.log10(abs(x)))) + n)

def str_fmt(x, n=2):
    " Format x into nice Latex rounding to n"
    if x<0.1:
        if x!=.01 and x!=.001 and x!=.0001 and x!=.00001:
            power = int(np.log10(round_to_n(x, 0)))-1
            mypower=int(math.floor(np.sign(x) * math.log10(abs(x))))
            f_SF = round_to_n(x, n) * pow(10, -power)
            f_SF = math.floor(x*10**(-mypower+2))/100
            return r"${} \, 10^{{ {} }}$".format(f_SF, mypower)
        else:
            power = int(np.log10(round_to_n(x, 0)))
            f_SF = round_to_n(x, n) * pow(10, -power)
            return r"${} \, 10^{{ {} }}$".format(f_SF, power)
    elif x>99:
        power = int(np.log10(round_to_n(x, 0)))
        f_SF = round_to_n(x, n) * pow(10, -power)
        return r"${} \, 10^{{ {} }}$".format(f_SF, power)
    else:
        return r"${}$".format(x)
    
def str_fmt_round_numb(x, n=1):
    " Format x into nice Latex rounding to n"
    if x<0.1:
        power = int(np.log10(round_to_n(x, 0)))-1
        f_SF = round_to_n(x, n) * pow(10, -power)
        return r"${} \, 10^{{ {} }}$".format(f_SF, power)
    elif x>99:
        power = int(np.log10(round_to_n(x, 0)))
        f_SF = round_to_n(x, n) * pow(10, -power)
        return r"${} \, 10^{{ {} }}$".format(f_SF, power)
    else:
        return r"${}$".format(x)

def str_fmt_name_fig(x, n=2):
    " Format x into nice text - rounding to n"
    if x<0.1:
        if x!=.01 and x!=.001 and x!=.0001 and x!=.00001:
            power = int(np.log10(round_to_n(x, 0)))-1
            mypower=int(math.floor(np.sign(x) * math.log10(abs(x))))
            f_SF = round_to_n(x, n) * pow(10, -power)
            f_SF = math.floor(x*10**(-mypower+2))/100
            return r"{}*10({})".format(f_SF, mypower)
        else:
            power = int(np.log10(round_to_n(x, 0)))
            f_SF = round_to_n(x, n) * pow(10, -power)
            return r"{}*10^({ })".format(f_SF, power)
    elif x>99:
        power = int(np.log10(round_to_n(x, 0)))
        f_SF = round_to_n(x, n) * pow(10, -power)
        return r"{}*10^({ })".format(f_SF, power)
    else:
        return r"{}$".format(x)
    


Example of hill coefficient computation (when we know the answer!)

In [3]:
n=30#hill coef
input=np.logspace(-10, 1, 50)
output=input**n/(1+input**n)



maxoutput=max(output)

approxloc09=np.abs(output-.9*maxoutput).argmin()#position in the array of approximate location
input09=np.linspace(input[approxloc09-1],input[approxloc09+1],100)#generate more output points near that approximate location 
output09=input09**n/(1+input09**n)
inversef=interp1d(output09,input09)
loc09=inversef(.9*maxoutput)

approxloc01=np.abs(output-.1*maxoutput).argmin()#position in the array of approximate location
input01=np.linspace(input[approxloc01-1],input[approxloc01+1],100)#generate more output points near that approximate location 
output01=input01**n/(1+input01**n)
inversef=interp1d(output01,input01)
loc01=inversef(.1*maxoutput)
    
hillcoef = math.log(81)/math.log(loc09/loc01)


## Covalent modification cycle switch

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC349147/pdf/pnas00662-0302.pdf

We start by choosing parameters and defining equations

In [4]:
###################################
# PARAMETERS
###################################

scale = 1#to be able to change units if needed

# Input of the pathway: FGF signaling via SOS : 
maxn = 100; #number of points 
sostotlist=np.logspace(-10, 1, num=maxn)*scale #logarithmic ploting
maxplots=5;
selectedsos=range(1,maxn,divmod(maxn,maxplots)[0])#we plot only n=maxplots plots 


#Output of the pathway: we look at the output of the system after t=tmax
tmax = 100
tmaxLONG = 100
 

#(* mean value for the parameters *)
WTV = 0.0023*scale
#(* small to have a small input in the cascade, cf Ferrell.. *)
rasgapV = .0002*scale
#(* we vary this paremeter in the code, so this value is not used *)
k11V = 1 #(* randomly chosen *)
k12V = 1
d11V = 1 #* randomly chosen *)
d12V = 1 #(* randomly chosen *)
a11V = 10000/scale #(* randomly chosen *)
a12V = 10000/scale #(* randomly chosen *)


###################################
# EQUATIONS
###################################

def equationsswitch(var,t,params):
    
    rasgdp, WE1, WsE2 = var
    WT, a11, a12, d11, d12, k11, k12, rasgap, sostot = params
    
     
    drasgdp = -a11*rasgdp*(sostot - WE1) + d11*WE1 + k12*WsE2
    dWE1 = a11*rasgdp*(sostot - WE1) - (d11 + k11)*WE1
    dWsE2 = a12*(WT - rasgdp - WE1 - WsE2)*(rasgap - WsE2) - (d12+k12)*WsE2
     
    return [drasgdp, dWE1, dWsE2]
    
###################################
# ZOO CONDITIONS?
###################################

sostotlist[maxn-1]/WTV<10**(-1)
rasgapV/WTV<10**(-1)
(k11V+d11V)/a11V/WTV<10**(-1)
(k12V+d12V)/a12V/WTV<10**(-1)
    
###################################
# INITIAL CONDTIONS
###################################
    
initialcond = [WTV,0,0]
    
###################################
# time array & array to store the output for various input levels of sos
###################################

tStop = tmax
tInc = 0.1
t = np.arange(0., tStop, tInc)



Then we look over sos (fgf) input values at fixed ephrin value + compute the corresponding Hill coefficient

In [5]:
###################################
# solve equations for various sos values (ephrin signaling)
###################################


max_erk_values = 10; #number of points 
erktotlist=np.logspace(-10, 1, num=max_erk_values)*scale #logarithmic ploting
hillcoeferk=[-1]

for kk in range(len(erktotlist)):
    rasgapV=erktotlist[kk]
    fig=plt.figure()
    storeerk=[-1]

    #to number plots    np.arange(1,math.floor(len(range(0,maxn,divmod(maxn,10)[0])))+1)
    plotnumber=0
    for i in range(1,maxn):

        psoln = odeint(equationsswitch, initialcond, t, args=([WTV, a11V, a12V, d11V, d12V, k11V, k12V, rasgapV,sostotlist[i]],))
        rasgtp=WTV-(psoln[:,0]+psoln[:,1]+psoln[:,2])

        ###################################
        # plot time series & output/input function
        ###################################
        if i in selectedsos:
             plotnumber=plotnumber+1

             plt.subplot(math.floor(len(selectedsos)/2)+1,2,plotnumber)
             plt.rc('text', usetex = True)
             plt.plot(rasgtp,label=r"$ \textnormal{rasgtp}$")
             plt.plot(np.ones(len(t))*WTV,label=r" \textnormal{Ras max}")
             plt.legend()
             plt.title(r'Temporal evolution of Ras-GTP for sos = {}'.format(str_fmt(sostotlist[i],n=2)))
             plt.yticks([0, WTV/4, WTV/2, 3*WTV/4, WTV],
               ['$0$', r'{}'.format(str_fmt(WTV/4)), r'{}'.format(str_fmt(WTV/2)), r'{}'.format(str_fmt(3*WTV/4)), r'{}'.format(str_fmt(WTV))])

         #is steady state reached?
        if (rasgtp[(len(rasgtp)-1)]-rasgtp[int(math.floor(.9*(len(rasgtp)-1)))])/WTV<10**(-3)==False:
                print("steady state not reached")
        # if yes, keep rasgtp level reached at ss
        storeerk=np.append(storeerk,[rasgtp[(len(rasgtp)-1)]])
        #end on the loop over sos values. 



    #remove first element of storeerk (it has no meaning)
    storeerk=storeerk[1:] 
    ymaxtick=round_to_n(max(storeerk),0)
    # !!! if you want to try to extrapolate add next line !!!!
    #f=interp1d(range(1,maxn),storeerk,kind='cubic')



    ###################################
    # compute hill coefficient 
    ###################################
    #
    output=storeerk
    input1=sostotlist[1:]
    maxoutput=max(output)
    #
    approxloc09=np.abs(output-.9*maxoutput).argmin()#position in the array of approximate location
    input09=np.linspace(input1[approxloc09-2],input1[approxloc09+2],100)#generate more output points near that approximate location 
    output09=[-1]#initiate the local output array
    #
    #
    for i in range(0,len(input09)):
        psoln = odeint(equationsswitch, initialcond, t, args=([WTV, a11V, a12V, d11V, d12V, k11V, k12V, rasgapV,input09[i]],))
        rasgtp=WTV-(psoln[:,0]+psoln[:,1]+psoln[:,2])
        output09=np.append(output09,[rasgtp[(len(rasgtp)-1)]])
    # 
    output09=output09[1:]
    inversef=interp1d(output09,input09)
    loc09=inversef(.9*maxoutput)
    #
    #
    approxloc01=np.abs(output-.1*maxoutput).argmin()#position in the array of approximate location
    input01=np.linspace(input1[approxloc01-2],input1[approxloc01+2],100)#generate more output points near that approximate location 
    output01=[-1]
    #
    #
    for i in range(0,len(input01)):
        psoln = odeint(equationsswitch, initialcond, t, args=([WTV, a11V, a12V, d11V, d12V, k11V, k12V, rasgapV,input01[i]],))
        rasgtp=WTV-(psoln[:,0]+psoln[:,1]+psoln[:,2])
        output01=np.append(output01,[rasgtp[(len(rasgtp)-1)]])
    # 
    output01=output01[1:]
    inversef=interp1d(output01,input01)
    loc01=inversef(.1*maxoutput)
    #
    hillcoef=math.log(81)/math.log(loc09/loc01)
    hillcoeferk=np.append(hillcoeferk,hillcoef)

    ###################################
    # add plot input/out 
    ###################################


    plt.subplot(math.floor(len(selectedsos)/2)+1,2,plotnumber+1)
    plt.plot(range(1,maxn),storeerk,label=r"output/input1")#,label="computed points"
    plt.legend(loc=4)
    plt.yticks([0, ymaxtick/4, ymaxtick/2, 3*ymaxtick/4, ymaxtick],
               ['$0$', r'{}'.format(str_fmt(ymaxtick/4)), r'{}'.format(str_fmt(ymaxtick/2)), r'{}'.format(str_fmt(3*ymaxtick/4)), r'{}'.format(str_fmt(ymaxtick))])
    plt.ylabel('output')
    plt.xlabel('input')
    plt.text(0,3*ymaxtick/4, r'Hill coefficient = {}'.format(round_to_n(hillcoef,2)), style='italic',
        bbox={'facecolor':'lightblue', 'alpha':0.5, 'pad':10})
    #plt.text(10,ymaxtick/4,r'Hill coef = {}'.format(hillcoef)))
    # !!! if you did an extrapolation and want to plot it add next 2 lines!!!!
    #plt.plot(selectedsos,f(selectedsos),label="extrapolation")
    #plt.legend()

    fig.savefig(r'full_figure_erk_{}.pdf'.format(str_fmt_name_fig(rasgapV)))      
    plt.close(fig)

hillcoeferk=hillcoeferk[1:]

fig=plt.figure()
plt.plot(erktotlist,hillcoeferk,label=r"output/input1")#,label="computed points"
#plt.legend(loc=4)
#plt.yticks([0, ymaxtick/4, ymaxtick/2, 3*ymaxtick/4, ymaxtick],
#          ['$0$', r'{}'.format(str_fmt(ymaxtick/4)), r'{}'.format(str_fmt(ymaxtick/2)), r'{}'.format(str_fmt(3*ymaxtick/4)), r'{}'.format(str_fmt(ymaxtick))])
plt.ylabel('hill coef')
plt.xlabel('erk')
plt.xscale('log')
#plt.text(10,ymaxtick/4,r'Hill coef = {}'.format(hillcoef)))
# !!! if you did an extrapolation and want to plot it add next 2 lines!!!!
#plt.plot(selectedsos,f(selectedsos),label="extrapolation")
#plt.legend()

fig.savefig(r'hillcoef_erk.pdf')      
plt.close(fig)

In [16]:
round_to_n(hillcoef,2)

1.9

In [6]:
var

NameError: name 'var' is not defined

## Stochasticity on ZOU

People have shown that, in small volume such as a single cell, the sharpness of the ZOU switch is not as big as in large volumes:
http://www.sciencedirect.com/science/article/pii/S0006349500763776

To do gillespie, use stochpy
http://pythonhosted.org/StochPy/userguide_doc.html#module-1-demo